<a href="https://colab.research.google.com/github/misbahsy/APMonitor-do/blob/master/DiscreteVariables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dynamic Control with Discrete Variables
One often encounters problems in which manipulated variables or adjustable parameters must be selected from among a set of discrete values. Examples of discrete variables include ON/OFF state, selection of different feed lines, and other variables that are naturally integers. Many dynamic optimization problems are discrete in nature.

In [0]:
#@title
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/RPKPjqdiX1c" frameborder="0" allowfullscreen></iframe>


<p class='vspace'>At first glance it might seem solving a discrete variable problem would be easier than a continuous problem. After all, for a variable within a given range, a set of discrete values within the range is finite whereas the number of continuous values is infinite. When searching for an optimum, it seems it would be easier to search from a finite set rather than from an infinite set.
</p>
<p class='vspace'>This is not the case, however. Solving discrete problems is harder than continuous problems. This is because of the combinatorial explosion that occurs in all but the smallest problems. For example if we have two variables which can each take 10 values, we have 10*10 = 100 possibilities. If we have 10 variables that can each take 10 values, we have 10^10 possibilities. Even with the fastest computer, it would take a long time to evaluate all of these.
</p>
<p class='vspace'>Discrete decision variables are those that have only certain levels or quantities that are acceptable at an optimal solution. Examples of discrete variables are binary (e.g. off/on or 0/1), integer (e.g. 4,5,6,7), or general discrete values that are not integer (e.g. 1/4 cm, 1/2 cm, 1 cm).
</p>
<div class='vspace'></div>



##Integer Variables
<p>Integer or binary variables are defined in the APMonitor Modeling Language by appending a variable name with <strong>int</strong>. An binary decision variable is an integer variable with bounds between 0 and 1.
</p>
<div class='vspace'></div><pre> ! Binary decision variable (0 or 1)
 Variables
   int_b &gt;=0 &lt;=1
</pre><p class='vspace'>The range of upper and lower bounds can be increased or decreased to any range to create a more general integer variable.
</p>
<div class='vspace'></div><pre> ! Integer decision variable (-5 to 10)
 Variables
   int_v &gt;=-5 &lt;=10
</pre><p class='vspace'>Nonlinear programming solvers (such as IPOPT) may not return an integer solution because they are designed for continuous variables. Mixed Integer Nonlinear Programming solvers (such as APOPT) are equipped to solve for binary or integer variables. Select the appropriate <a class='urllink' href='https://apmonitor.com/wiki/index.php/Main/OptionApmSolver' rel='nofollow'>solver option</a> to either find an initial solution without integer variables or an integer solution. It is sometimes desirable to find a non-integer solution first because of the often significant reduction in computation time without the integer variables.
</p>




##Exercise 1

<p><strong>Objective:</strong> Solve a discrete optimization problem with the branch and bound technique. <em>Estimated time: 1 hour.</em>
</p>
<div class='vspace'></div><ul><li><a class='urllink' href='http://apmonitor.com/do/uploads/Main/minlp_worksheet.pdf' rel='nofollow'>Branch and Bound Exercise (pdf)</a>
</li></ul><div class='vspace'></div><div><img src='http://apmonitor.com/do/uploads/Main/bnb_contour.png' alt='' title='' /></div>
<div class='vspace'></div>


##Solution

<p class='vspace'>The integer problem is solved with <a class='urllink' href='https://gekko.readthedocs.io/en/latest/' rel='nofollow'>Python GEKKO</a>. The option <strong>integer</strong>=True is used to switch the variable from continuous to integer form. The APOPT solver is required to solve problem with integer variables but other solvers such as IPOPT can provide a relaxed solution where the integer conditions are not enforced.
</p>
<div><img src='http://apmonitor.com/do/uploads/Main/download.png' alt='' title='' /> <a class='urllink' href='http://apmonitor.com/do/uploads/Main/discrete_optimization_source.zip' rel='nofollow'>Discrete Optimization Source Files</a></div>
<div class='vspace'></div>

In [0]:
#@title Solution with GEKKO
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
pipmain(['install','gekko'])

from gekko import GEKKO
m = GEKKO() # create GEKKO model
# create integer variables
x1 = m.Var(integer=True,lb=-1,ub=1) 
x2 = m.Var(integer=True,lb=-1,ub=2)
m.Obj(4*x1**2-4*x2*x1**2+x2**2+x1**2-x1+1)
m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0]))
print('x2: ' + str(x2.value[0]))


In [0]:
#@title
%%html
<iframe width="560" height="315" src="//www.youtube.com/embed/XhqteZIydT0?rel=0" frameborder="0" allowfullscreen></iframe>


##Exercise 2
<p>Design a thermostat for a home that has a furnace with On/Off control. The home is initially unheated in 40 degF weather. When the heater is turned on, the temperature in the home reaches 63% of the steady state value in 120 minutes, starting from 40 degF. If left on indefinitely, the home would eventually reach 100 degF. Optimize the heater response for the first 2 1/2 hours starting from a cold home at 40 degF. The target range for the home is between 68 and 70 degF.
</p>
<div class='vspace'></div><div><img src='http://apmonitor.com/do/uploads/Main/download.png' alt='' title='' /> <a class='urllink' href='http://apmonitor.com/do/uploads/Main/thermostat_python_matlab.zip' rel='nofollow'>Optimized Home Furnace Source Files</a></div>

##Solution

In [0]:
#@title Solution with GEKKO

try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
pipmain(['install','gekko'])

from gekko import GEKKO
import matplotlib.pyplot as plt

m = GEKKO() # create GEKKO model
m.time = [0,1,2,4,8,12,16,20,25,30,35,40,45,50,\
          55,60,65,70,75,80,85,90,95,100,105,110,\
          115,120,125,130,135,140,145,150]

# change solver options
m.solver_options = ['minlp_gap_tol 1.0e-2',\
                    'minlp_maximum_iterations 10000',\
                    'minlp_max_iter_with_int_sol 5000']
m.options.SOLVER = 1
m.options.IMODE = 6

# parameters
Kp = 60
taup = 120
T0 = 40

# create heater binary variable
u = m.MV(integer=True,lb=0,ub=1)
u.dcost = 0.1
u.status = 1

# controlled variable (temperature)
T = m.CV(value=T0)
T.SPHI = 68
T.SPLO = 70
T.STATUS = 1
T.TR_INIT = 0

# first order equation
m.Equation(taup * T.dt() == -(T-T0) + Kp * u)

m.solve() # solve MILP

# plot solution
plt.subplot(2,1,1)
plt.plot(m.time,T,'r-',label='Temperature')
plt.plot([0,m.time[-1]],[T.SPHI,T.SPHI],'k:',label=r'$T_{sphi}$')
plt.plot([0,m.time[-1]],[T.SPLO,T.SPLO],'k:',label=r'$T_{splo}$')
plt.ylabel('T (degF)')
plt.legend()
plt.subplot(2,1,2)
plt.plot(m.time,u,'b.-',label='Furnace (On/Off)')
plt.ylabel('Heat (On/Off)')
plt.xlabel('Time (min)')
plt.legend()
plt.show()

In [0]:
#@title
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/2XC6mpop_j4" frameborder="0" allowfullscreen></iframe>


##Exercise 3</h4>
<p>An example problem for Mixed Integer Optimal Control is the Lotka Volterra fishing problem. It finds an fishing strategy over a 12 year horizon to equilibrate the predator as prey fish to a minimum steady state value. The Lotka Volterra equations for a predator-prey system have an additional equation to introduce fishing by man and with constants `c_0=0.4` and `c_1=0.2`.
</p>
<p class='vspace'>$$\min_{x,w} x_2 \left( t_f \right)$$
</p>
<p class='vspace'>$$\mathrm{s.t.} \quad \frac{dx_0}{dt} = x_0 - x_0 x_1 - c_0 x_0 w$$
$$\quad \quad  \frac{dx_1}{dt} = -x_1 + x_0 x_1 - c_1 x_1 w$$
$$\quad \quad  \frac{dx_2}{dt} = \left(x_0-1\right)^2 + \left(x_1-1\right)^2$$
$$\quad \quad x(0) = (0.5,0.7,0)^T$$
$$\quad \quad w(t) \in {{0,1}}$$
</p>
<p class='vspace'>The optimal solution has a final objective of 1.344.
</p>
<div class='vspace'></div><div><img width='550px' src='http://apmonitor.com/do/uploads/Main/lotka_volterra_fishing.png' alt='' title='' /> </div>

<div class='vspace'></div><div><img src='http://apmonitor.com/do/uploads/Main/download.png' alt='' title='' /> <a class='urllink' href='http://apmonitor.com/do/uploads/Main/lotka_volterra_fishing.zip' rel='nofollow'>Download Solution Files in MATLAB and Python</a></div>



In [0]:
#@title Soultion with GEKKO

try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
pipmain(['install','gekko'])

import numpy as np
import matplotlib.pyplot as plt
from gekko import GEKKO

m = GEKKO() # create GEKKO model

# Add 0.01 as first step
# 0,0.01,0.1,0.2,0.3,...11.9,12.0)
m.time = np.insert(np.linspace(0,12,121),1,0.01)

# change solver options
m.solver_options = ['minlp_gap_tol 0.001',\
                    'minlp_maximum_iterations 10000',\
                    'minlp_max_iter_with_int_sol 100',\
                    'minlp_branch_method 1',\
                    'minlp_integer_tol 0.001',\
                    'minlp_integer_leaves 0',\
                    'minlp_maximum_iterations 200']

c0 = 0.4 
c1 = 0.2

last = m.Param(np.zeros(122))
last.value[-1] = 1

x0 = m.Var(value=0.5,lb=0)
x1 = m.Var(value=0.7,lb=0)
x2 = m.Var(value=0.0,lb=0)
w = m.MV(value=0,lb=0,ub=1,integer=True)
w.STATUS = 1

m.Obj(last*x2)

m.Equations([x0.dt() == x0 - x0*x1 - c0*x0*w,\
             x1.dt() == - x1 + x0*x1 - c1*x1*w,\
             x2.dt() == (x0-1)**2 + (x1-1)**2])

m.options.IMODE = 6
m.options.NODES = 3
m.options.SOLVER = 1
m.options.MV_TYPE = 0
m.solve()

plt.figure(1)
plt.step(m.time,w.value,'r-',label='w (0/1)')
plt.plot(m.time,x0.value,'b-',label=r'$x_0$')
plt.plot(m.time,x1.value,'k-',label=r'$x_1$')
plt.plot(m.time,x2.value,'g-',label=r'$x_2$')
plt.xlabel('Time')
plt.ylabel('Variables')
plt.legend(loc='best')
plt.show()


<div class='vspace'></div><h4>Additional Resources</h4>
<ul><li><a class='urllink' href='https://mintoc.de' rel='nofollow'>Mixed Integer Optimal Control Problems (mintOC)</a>
</li><li><a class='urllink' href='https://minlp.org' rel='nofollow'>CMU-IBM Cyber-Infrastructure for MINLP</a>
</li></ul>
</div>